<a href="https://colab.research.google.com/github/Atharvaahirrao/Practical/blob/main/dagster_yfinance_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagster dagster-webserver yfinance pandas numpy scikit-learn matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.8/205.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/2

In [2]:
import os
import yfinance as yf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from dagster import asset, Failure, materialize

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error


In [27]:
from dagster import asset
import pandas as pd
import numpy as np

@asset
def stock_data():
    print("📊 Loading Yahoo Finance (simulated) stock data")

    rows = 300  # change this to prove partial rerun

    df = pd.DataFrame({
        "Date": pd.date_range(start="2019-01-01", periods=rows),
        "Open": np.random.normal(149, 10, rows),
        "High": np.random.normal(151, 10, rows),
        "Low": np.random.normal(148, 10, rows),
        "Close": np.random.normal(150, 10, rows),
        "Volume": np.random.randint(1_000_000, 5_000_000, rows)
    })

    return df


In [22]:
!rm aapl_stock.csv


rm: cannot remove 'aapl_stock.csv': No such file or directory


In [4]:
@asset
def eda(stock_data: pd.DataFrame):
    plt.figure(figsize=(10,4))
    sns.lineplot(x="Date", y="Close", data=stock_data)
    plt.title("Apple Stock Closing Price")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.show()
    return "EDA completed"


In [5]:
@asset
def features(stock_data: pd.DataFrame):
    df = stock_data.copy()

    df["Returns"] = df["Close"].pct_change()
    df["MA_10"] = df["Close"].rolling(10).mean()
    df["MA_50"] = df["Close"].rolling(50).mean()

    df.dropna(inplace=True)

    if df.empty:
        raise Failure("Feature engineering produced empty dataset")

    return df


In [6]:
@asset
def train_test_data(features: pd.DataFrame):
    if features.empty:
        raise Failure("Features dataset is empty")

    X = features[["Returns", "MA_10", "MA_50"]]
    y = features["Close"]

    return train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
@asset
def decision_tree(train_test_data):
    X_train, X_test, y_train, y_test = train_test_data

    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    print("Decision Tree MSE:", mse)
    return mse


In [8]:
@asset
def random_forest(train_test_data):
    X_train, X_test, y_train, y_test = train_test_data

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    print("Random Forest MSE:", mse)
    return mse


In [9]:
@asset
def gradient_boost(train_test_data):
    X_train, X_test, y_train, y_test = train_test_data

    model = GradientBoostingRegressor(random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    print("Gradient Boosting MSE:", mse)
    return mse


In [28]:
materialize([
    stock_data,
    features,
    train_test_data,
    decision_tree,
    random_forest,
    gradient_boost
])


2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - RUN_START - Started execution of run for "__ephemeral_asset_job__".
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - ENGINE_EVENT - Executing steps in process (pid: 1589)
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - RESOURCE_INIT_STARTED - Starting initialization of resources [io_manager].
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - RESOURCE_INIT_SUCCESS - Finished initialization of resources [io_manager].
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - LOGS_CAPTURED - Started capturing logs in process (pid: 1589).
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 517

📊 Loading Yahoo Finance (simulated) stock data
Decision Tree MSE: 73.84207898127181


2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - gradient_boost - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - gradient_boost - Writing file at: /tmp/tmp_2t1km2b/storage/gradient_boost using PickledObjectFilesystemIOManager...
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - gradient_boost - ASSET_MATERIALIZATION - Materialized value gradient_boost.
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - gradient_boost - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2026-01-27 16:00:25 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - gradient_boost - STEP_SUCCESS - Finished execu

Gradient Boosting MSE: 33.60076273469297


2026-01-27 16:00:26 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - random_forest - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2026-01-27 16:00:26 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - random_forest - Writing file at: /tmp/tmp_2t1km2b/storage/random_forest using PickledObjectFilesystemIOManager...
2026-01-27 16:00:26 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - random_forest - ASSET_MATERIALIZATION - Materialized value random_forest.
2026-01-27 16:00:26 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - random_forest - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2026-01-27 16:00:26 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 5176af65-b5e0-4b43-b93d-d37fa566b0dd - 1589 - random_forest - STEP_SUCCESS - Finished execution of

Random Forest MSE: 35.330594958231316


In [ ]:
materialize([
    stock_data,
    features,
    train_test_data,
    decision_tree,
    random_forest,
    gradient_boost
])


2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - RUN_START - Started execution of run for "__ephemeral_asset_job__".
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - ENGINE_EVENT - Executing steps in process (pid: 669)
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - RESOURCE_INIT_STARTED - Starting initialization of resources [io_manager].
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - RESOURCE_INIT_SUCCESS - Finished initialization of resources [io_manager].
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - LOGS_CAPTURED - Started capturing logs in process (pid: 669).
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b

✅ Using cached stock data


2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - features - Writing file at: /tmp/tmpzi_jtx40/storage/features using PickledObjectFilesystemIOManager...
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - features - ASSET_MATERIALIZATION - Materialized value features.
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - features - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - features - STEP_SUCCESS - Finished execution of step "features" in 150ms.
2026-01-21 14:49:16 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - train_test_data - STEP_START - Started execution of step "train_test_data".
2026-01-21 14:49:16 +

Decision Tree MSE: 125.65272570759015


2026-01-21 14:49:17 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - gradient_boost - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2026-01-21 14:49:17 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - gradient_boost - Writing file at: /tmp/tmpzi_jtx40/storage/gradient_boost using PickledObjectFilesystemIOManager...
2026-01-21 14:49:17 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - gradient_boost - ASSET_MATERIALIZATION - Materialized value gradient_boost.
2026-01-21 14:49:17 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - gradient_boost - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2026-01-21 14:49:17 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - gradient_boost - STEP_SUCCESS - Finished execution

Gradient Boosting MSE: 73.98078099613386


2026-01-21 14:49:18 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - random_forest - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2026-01-21 14:49:18 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - random_forest - Writing file at: /tmp/tmpzi_jtx40/storage/random_forest using PickledObjectFilesystemIOManager...
2026-01-21 14:49:18 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - random_forest - ASSET_MATERIALIZATION - Materialized value random_forest.
2026-01-21 14:49:18 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - random_forest - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2026-01-21 14:49:18 +0000 - dagster - DEBUG - __ephemeral_asset_job__ - 8c0ece6f-b0b3-4160-a01c-2e3d36d8b884 - 669 - random_forest - STEP_SUCCESS - Finished execution of ste

Random Forest MSE: 70.62240149096421
